In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.model_selection import train_test_split

/kaggle/input/melbourne-housing-snapshot/melb_data.csv


In [2]:
data = pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [3]:
y=data['Price']
X= data.drop(['Price'],axis=1)
X_train_full, X_valid_full, y_train,y_valid = train_test_split(X,y, train_size=0.8,test_size=0.2,random_state=0)

Seprating categorical and numerical columns:
for this I used 'X' in the list comprehension instead of 'X_train_full' and there were no differences in the result! but then check with chat-gpt.


In [4]:
# categorical_cols = [cname for cname in X if X[cname].dtype=='object' and X[cname].nunique()<10]
# categorical_cols
# numerical_cols= [cname for cname in X if X[cname].dtype in ['int','float']]
# numerical_cols

If you use X instead of X_train_full, you are checking the same conditions but on the entire dataset (both training and validation sets combined).
In practice, this might not make a significant difference because the validation set is typically similar to the training set. However, it's a good practice to base such decisions on the training set alone to avoid data leakage and ensure that your model is generalizable to new, unseen data.
In summary, while using X might give you the same result in many cases, it is a good practice to use the training set (X_train_full) when making decisions about feature engineering or preprocessing steps to avoid unintentional data leakage and ensure a robust modeling process.
Data Leakage in Preprocessing:

Data leakage can occur when information from the validation or test set is inadvertently used during the preprocessing phase. This can lead to optimistic performance estimates during model development but may result in poor generalization to new, unseen data.

In [5]:
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [6]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [7]:
X_train['Regionname'].nunique()

8

***What is the meaning of(strategy ='constant') in SimpleImputer?***

This means that missing values in the numeric columns will be replaced with a constant value.

By default, the constant value used is 0.0. If you want to specify a different constant value, you can do so by setting the fill_value parameter. For example, if you want to replace missing values with the constant value 99:


numerical_imputer = SimpleImputer(strategy='constant', fill_value=99)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')
numerical_transformer


SimpleImputer(strategy='constant')

In [9]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent' )),
                                          ('onehot',OneHotEncoder(handle_unknown ='ignore'))])
categorical_transformer

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Pipeline definition by chat-gpt:
A pipeline is like a workflow that helps you organize and automate different steps in a machine learning process.
In this workflow, you have a series of tasks (steps) that need to be done, like preparing the data and building a predictive model.
Each task is either something that prepares the data (a transformer) or something that makes predictions (an estimator).
Transformers are like data handlers; they transform or preprocess the data. They have methods called fit and transform.
Estimators are like models; they make predictions based on the data they have learned from. They have a method called fit to learn from the data.
So, in a pipeline:

The first tasks are transformers that prepare your data.
The last task is an estimator that makes predictions based on the prepared data.
All tasks in between are transformers, ensuring the data is ready for the final prediction.


# Question:
**so this pipeline does not have estimator step. right? is this conclusion correct?"there are pipelines that do transformation steps, hence they do not have estimators." **
# Answer:
ChatGPT:

Yes, your conclusion is correct. In the provided categorical_transformer pipeline:


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
There is no final step that involves making predictions (an estimator). This pipeline is focused on transforming (preprocessing) the data, specifically handling missing values and one-hot encoding categorical variables.

Your generalization is also accurate:
"There are pipelines that do transformation steps, hence they do not have estimators."

Some pipelines are designed solely for data preprocessing and do not include an estimator for making predictions. Estimators are typically found in pipelines where the final step involves building a model to make predictions.



In [10]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [12]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855
